In [1]:
from ebooklib import epub
from bs4 import BeautifulSoup
from copy import deepcopy
import bs4,ebooklib
import pandas as pd

# 创建一个epub对象
book = epub.read_epub('/Users/minkexiu/Desktop/图解周易大全(全译插图版) (图解经典系列) (贺华章 [贺华章]) (Z-Library).epub')

/opt/anaconda3/envs/ml12/lib/python3.12/site-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/opt/anaconda3/envs/ml12/lib/python3.12/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [2]:
def deal_with_yaoci(sup):
    yaoci_s = []
    for xx in sup:
        final_rst = ""
        for x in xx.contents:
            if x in ["\n", "\r\n"]:
                continue
            elif isinstance(x, bs4.element.Tag):
                final_rst += "".join([y for y in x.contents if isinstance(y, str)])
            else:
                final_rst += x.replace("\r", "").replace("\n", "")
        final_rst = final_rst.strip()
        if final_rst == "":
            continue
        yaoci_s.append(final_rst)
    return yaoci_s
    
    # yaoci_s = []
    # for xx in sup:
    #     final_rst = ""
    #     for x in xx.contents:
    #         if x in ["\n", "\r\n"]:
    #             continue
    #         elif isinstance(x, bs4.element.Tag):
    #             final_rst += "".join(x.contents)
    #         else:
    #             final_rst += x.replace("\r", "").replace("\n", "")
    #     yaoci_s.append(final_rst)
    # return yaoci_s
    
def multiply(lst):
    '''每一个元素重复6次。因为64卦是6个爻嘛。'''
    rst = []
    for i in lst:
        rst.extend([i] * 6)
    return rst

import re
def drop_dup_keep_order(lst):
    rst = []
    for l in lst:
        if l not in rst:
            rst.append(l)
    return rst

def parse_soup(soup):
    # try: 
    yaoci = deal_with_yaoci(
        soup.select('p[class^="_0原文段前3"]') 
    )
    if '文言曰 ：坤至柔而动也刚，至静而德方，后得主而有常，含万物而化光。坤道其顺乎？承天而时行。' in yaoci:
        new_yc = []
        for yc in yaoci:
            new_yc.append(re.sub(r'\u3000', '', yc))
        new_yc.remove('文言曰 ：坤至柔而动也刚，至静而德方，后得主而有常，含万物而化光。坤道其顺乎？承天而时行。')
        yaoci = new_yc
    if '象曰： 即鹿无虞，以从禽也。君子舍之，往吝穷也。' in yaoci:
        yaoci.remove('象曰： 即鹿无虞，以从禽也。君子舍之，往吝穷也。')
    
    xiangci = drop_dup_keep_order(
        [
            re.sub(r'\s', '', x.replace("[4]", "")) for x in deal_with_yaoci(
                soup.select('p[class^="_0原文段后1"],p[class="_0原文 ParaOverride-7"]') #
            ) if x.startswith('象曰')
        ]
    )
    if "初六： 拔茅，茹以其汇，贞吉，亨。" in yaoci:
        xiangci = drop_dup_keep_order(
            [
                re.sub(r'\s', '', x.replace("[4]", "")) for x in deal_with_yaoci(
                    soup.select('p[class^="_0原文段后1"]') #
                ) if x.startswith('象曰')
            ]
        )
    if '象曰：用六永贞，以大终也。' in xiangci:
        xiangci.remove('象曰：用六永贞，以大终也。')
        xiangci.insert(8, '象曰： 即鹿无虞，以从禽也。君子舍之，往吝穷也。')
    
    if '象曰：天下有山，遁。君子以远小人，不恶而严。' in xiangci:
        xiangci.remove('象曰：天下有山，遁。君子以远小人，不恶而严。')
    if '象曰：风雷，益；君子以见善则迁，有过则改。' in xiangci:
        xiangci.remove('象曰：风雷，益；君子以见善则迁，有过则改。')
    if '象曰：有孚发若，信以发志也。' in xiangci:
        yaoci = deal_with_yaoci(
            soup.select('p[class^="_0原文段前3后1"]') 
        )
        xiangci.insert(26, "象曰：丰其沛，不可大事也。折其右肱，终不可用也。")
    
    # tuanci_fanyi = []
    # jingwen_fanyi = []
    # xiangci_fanyi = []
    # for xx in soup.select('p[class="_000000正文"]'):
    #     hhd = xx.select('span') 
    #     rst = ""
    #     for x in hhd:
    #         for _ in x.contents:
    #             rst+=str(_)
    #     if "彖辞的意思是" in rst:
    #         tuanci_fanyi.append(rst)
    #     if "经文意思是" in rst:
    #         jingwen_fanyi.append(rst)
    #     if "象辞的意思是" in rst:
    #         xiangci_fanyi.append(rst)
    
    zhengwen = deal_with_yaoci(soup.select('p[class^="_000000正文"]'))
    if '象曰：有孚发若，信以发志也。' in xiangci:
        zhengwen = deal_with_yaoci(soup.select('p[class="正文"]'))
    tuanci_fanyi = [x for x in zhengwen if x.startswith("彖辞的意思是")]
    jingwen_fanyi = [x for x in zhengwen if x.startswith("经文意思是")]
    if  '经文意思是：拉着车轮，守正道吉祥。' in jingwen_fanyi:
        jingwen_fanyi.insert(45, "经文意思是：没有渡过河，征伐有凶险，有利于跋涉大川。")
    
    xiangci_fanyi = [x for x in zhengwen if x.startswith("象辞的意思是")]
    if '象辞的意思是：中正之道行事没有灾难，是因为九五居中但却还未光大。' in xiangci_fanyi:
        xiangci_fanyi.insert(20, "“上六”是要被决断的小人，在被穷追不舍时大声呼号，也无人理会，难逃凶险。")
    
    print(len(yaoci), len(xiangci) , len(jingwen_fanyi) , len(xiangci_fanyi), len(tuanci_fanyi))
    
    guaci_fanyi = [ctt for idx, ctt in enumerate(jingwen_fanyi) if not(idx % 7)]
    
    gua_xiangci_fanyi = [ctt for idx, ctt in enumerate(xiangci_fanyi) if not(idx % 7)]
    
    # gua = [xxx.contents[0] for xxx in soup.select('font[color="#C56F17"]')]
        
    print(len(yaoci))
        
    return pd.DataFrame(
        {
            # "卦名": multiply(gua),
            "卦辞翻译": multiply(guaci_fanyi),
            "彖辞翻译": multiply(tuanci_fanyi),
            "象辞翻译": multiply(gua_xiangci_fanyi),
            "爻辞": yaoci,
            "爻辞翻译": [ctt for idx, ctt in enumerate(jingwen_fanyi) if idx % 7],
            "小象辞": xiangci,
            "小象辞翻译": [ctt for idx, ctt in enumerate(xiangci_fanyi) if idx % 7],
        }
    )
    # except:
    #     # print("failed")
    #     return pd.DataFrame()

In [3]:
yaoci = [
    '初九曰 ：“潜龙勿用。”',
    '九二曰 ：“见龙在田，利见大人。”',
    '九三曰 ：“君子终日乾乾，夕惕若厉，无咎。”',
    '九四： “或跃在渊，无咎。”',
    '九五曰 ：“飞龙在天，利见大人。”',
    '上九曰 ：“亢龙有悔。”【用九】见群龙无首，吉。',
    
    '初六 履霜，坚冰至。',
    '六二 直方大，不习无不利。',
    '六三 含章可贞，或从王事，无成有终。',
    '六四 括囊，无咎无誉。',
    '六五 黄裳元吉。',
    '上六 龙战于野，其血玄黄。【用六】利永贞',
    
    '初九： 磐桓，利居贞，利建侯。',
    '六二： 屯如邅如，乘马班如。匪寇婚媾，女子贞不字，十年乃字。',
    '六三： 即鹿无虞，唯入于林中，君子几不如舍，往吝。',
    '六四： 乘马班如，求婚媾，往吉，无不利。',
    '九五： 屯其膏，小贞吉，大贞凶。',
    '上六： 乘马班如，泣血涟如。',
    
    '初六： 发蒙，利用刑人，用说桎梏，以往吝。',
    '九二： 包蒙吉；纳妇吉；子克家。',
    '六三： 勿用取女；见金夫，不有躬，无攸利。',
    '六四： 困蒙，吝。',
    '六五： 童蒙，吉。',
    '上九： 击蒙；不利为寇，利御寇。',
    
    '初九： 需于郊。利用恒，无咎。',
    '九二： 需于沙。小有言，终吉。',
    '九三：需于泥，致寇至。',
    '六四： 需于血，出自穴。',
    '九五： 需于酒食，贞吉。',
    '上六： 入于穴，有不速之客三人来，敬之终吉。'
]

jingwen_fanyi = [
    "【古解作】元，始也；亨，通也；利，和也；贞，正也。【今解作】当你满怀信心开始做一件事情时，谁能够阻止你的行为呢？谁也阻止不了，这就叫“元亨”。当你满怀信心地做一件事情时，是否能够持之以恒，坚持到底，这就需要“利贞”。",
    '这句经文的意思是：“身居下位，时机还没有成熟，所以应当像潜藏的龙一样不要施展你的才干。”',
    '此句经文的意思是：“龙出现在田野之上，有利于见到大人物来发展自己。”',
    '这句经文的意思是：“君子终日自强不息，每天晚上对自己进行深刻反省，保持警惕的心，检查自己的失误之处，才不会给自己带来灾难。”',
    '这句经文的意思是：“根据不同时机，可以一跃九天，也可以复沉于渊，都不会有什么灾难。”',
    '这一句经文的意思是：“飞龙遨游于天际，有利于见到大人物。”',
    '这一句经文的意思是：“龙向上飞得太高了，便会有后悔的事情发生。”【用九】出现一群龙而没有首领，而每条龙都能各尽其职，这很吉祥。',
    
    "先解释卦辞，将其译成现代文便是：坤卦，初始亨通，利于像母马那样保持温顺的德行。君子有所行动，起初会迷失方向，后得到主人，到西南方向可以得到朋友，在东北方向则丧失朋友。安于正道则会吉祥。",
    "这句爻辞的意思是，当脚踩到霜的时候，应该明白结冰的日子快到了。",
    "这句经文的意思是：正直、方正、大方，不反复没有什么不利的。",
    "经文意思是：蕴涵美德可以坚守正道，或者效力于君王，虽然没有成就但却有好的结果。",
    "经文意思是：将口袋的口束紧，没有灾难也没有荣誉。",
    "经文意思是：黄色的下衣，大吉大利。",
    "经文意思是：与龙在旷野厮杀，旷野到处是青与黄色的血迹。【用六】利于永远坚守正道。",
    
    '经文意思是：屯卦，天始亨通，有利于坚守正道。不要到别处求取功名。利于在自己的地盘上建立功业。',
    '经文意思是：像大石与木柱一样坚定，有利于居守正道，宜于树立王侯的威信。',
    '经文意思是：坎坷难于行进，骑在马上盘旋前进。不是贼寇，而是来求婚的人。女子坚守正道不出嫁，十年后才出嫁。',
    '经文意思是：追逐野鹿却没有猎官做向导，只有迷入山林之中。君子胸怀机智，不如舍弃，再往前走会有灾难。',
    '经文意思是：骑马团团打转，前去求婚，前进吉祥，没有不利的后果。',
    '经文意思是：屯聚云雨，雨小则吉祥，雨大则凶。',
    '经文意思是：骑在马上艰难行走，泣血涟涟，泪如雨下。',
    
    '经文意思是：蒙卦，亨通。不是我要去求蒙昧的儿童，而是蒙童来求我。初次占筮就告诉他。两次三次占筮就亵渎了神灵，对于亵渎神灵的就不能告诉他，利于固守正道。',
    '经文意思是：蒙昧的初期，利于用刑法惩治坏人，给冤屈者脱去枷锁，否则便会有忧吝。',
    '经文意思是：包容蒙昧，吉祥。娶妇吉祥。儿子可以持家了。',
    '经文意思是：不要娶这个女子，她见到有钱的男人就失去了自身的体统，没有什么好处。',
    '经文意思是：被困在蒙昧中，有忧吝。',
    '经文意思是：儿童的蒙昧，吉祥。',
    '经文意思是：用武力对付蒙昧，不利于追杀贼寇，利于防御贼寇。',
    
    '经文意思是：需卦，有诚信，光明而亨通，坚守正道吉祥。有利于渡过大川险阻。',
    '经文意思是：在郊外等待，有利于持恒，没有灾难。',
    '经文意思是：在沙滩上等待，有些小的口舌是非，最终是吉祥的。',
    '经文意思是：在淤泥中等待，导致贼寇来到。',
    '经文意思是：在忧患中等待，从洞穴中逃出。',
    '经文意思是：在酒食宴乐中等待，守正道则吉祥。',
    '经文意思是：进入洞穴，有不请自来的三位客人来到，尊敬他们，最终会得到吉祥。'
]
len(yaoci), len(jingwen_fanyi)

(30, 35)

In [4]:
xiangci = [
    '（小）象曰：潜龙勿用，阳在下也。',
    '见龙在田，德施普也。',
    '终日乾乾，反复道也。',
    '或跃在渊，进无咎也。',
    '飞龙在天，大人造也。',
    '亢龙有悔，盈不可久也。【用九】用九，天德不可为首也。',
    
    '象曰：履霜坚冰，阴始凝也。驯致其道，至坚冰也。',
    '象曰：六二之动，直以方也。不习无不利，地道光也。',
    '象曰：含章可贞；以时发也。或从王事，知光大也。',
    '象曰：括囊无咎，慎不害也。',
    '象曰：黄裳元吉，文在中也。',
    '象曰：龙战于野，其道穷也。【用六】用六永贞，以大终也。',
    
    '象曰：虽磐桓，志行正也。以贵下贱，大得民也。',
    '象曰：六二之难，乘刚也。十年乃字，反常也。',
    '象曰： 即鹿无虞，以从禽也。君子舍之，往吝穷也。',
    '象曰：求而往，明也。',
    '象曰：屯其膏，施未光也。',
    '象曰：泣血涟如，何可长也。',
    
    '象曰：利用刑人，以正法也。',
    '象曰：子克家，刚柔接也。',
    '象曰：勿用取女，行不顺也。',
    '象曰：困蒙之吝，独远实也。',
    '象曰：童蒙之吉，顺以巽也。',
    '象曰：利用御寇，上下顺也。',
    
    '象曰：需于郊，不犯难行也。利用恒，无咎；未失常也。',
    '象曰：需于沙，衍在中也。虽小有言，以终吉也。',
    '象曰：需于泥，灾在外也。自我致寇，敬慎不败也。',
    '象曰：需于血，顺以听也。',
    '象曰：酒食贞吉，以中正也。',
    '象曰：不速之客来，敬之终吉。虽不当位，未大失也。'
]

xiangci_fanyi = [
    '（大）象译文：天体运行刚健而永不停息，君子应当效法天道，自强不息。',
     '初九之所以“潜龙勿用”，是因为此阳爻的位置处于最下面的位置。',
     '九二的“见龙在田”，指的是九二阳爻已来到地上，可以发挥它的养生万物之德了。',
     '九三的“终日乾乾”，即反复运行，永不停息的意思。',
     '九四的“或跃在渊”，是说前进（或归隐）不会有灾难的意思。',
     '九五的“飞龙在天”，是指大人物将会出现而大有作为的意思。',
     '上九的“亢龙有悔”，是说物不可以终盛，会盛极而衰，阳极而阴生之意。【用九】阳刚之德不会自以为是，显示自己首领的地位。',
    
    "【自己整理】大地的气势柔顺、温和，以深厚的道德负载万物。",
    "象辞的意思是：履霜而知冰，这是因为阴气始生的缘故，随着阴气的不断加重，天寒冰冻的冬天就来了。",
    "象辞对这句经文的解释是：六二为动爻，正直而方正，不重复没有不利的，这是因为地道的广大啊。",
    "象辞对经文的解释是：含章可贞，是为了等待时机。为君王做事是因为智慧广大。",
    "象辞对经文的解释是，扎起口袋不会有灾难，是谨慎能使自己免于灾害。",
    "象传对经文的解释是：黄裳之所以大吉，是因为六五有温文之德并且守于中道。",
    "象传对经文的解释是：与龙战于野，是地道走到了穷途末路的缘故。【用六】用六的永贞，可以使坤卦的臣道得到大的善终。",
    
    '象辞的意思是：上卦为云，下卦为雷，这便是屯卦的卦象。君子应当从中得到启示，努力经营发展自己的事业。',
    '象辞的意思是：虽然徘徊难进，但前进的心愿符合正道。以高贵的地位平易近人地接近下贱的人，会广泛得到民众的拥护。',
    '象辞的意思是：六二爻的艰难，是由于乘驾在刚爻之上的缘故。十年才能生育，是返回常规。',
    '象辞的意思是：追逐野鹿却没有猎官做向导，只能跟着野兽跑。君子弃而不追，是因为前往会被困住。',
    '象辞的意思是：为了求婚而前往，是明智的选择。',
    '象辞的意思是：屯积云雨，是还没有广大施舍之意。',
    '象辞的意思是：哭得血泪满面，这样的状况怎么能会长久呢。',
    
    '象辞的意思是：山下流出清泉，这是蒙卦的卦象。君子效法此卦以果敢的行为来培养自己的道德。',
    '象辞的意思是：利于用刑法惩治坏人，是为了加强人们的法制观念。',
    '象辞的意思是：儿子持家，是刚爻与柔爻相接的缘故。',
    '象辞的意思是：不要娶这个女子，是因为这个女子不具备女人的柔顺之德。',
    '象辞的意思是：受困于蒙昧之中而忧吝，是因为六四爻独自远离充实的阳爻的缘故。',
    '象辞的意思是：儿童的蒙昧之所以吉祥，是因为他柔顺而谦逊。',
    '象辞的意思是：利于防御贼寇，是因为上下顺从的缘故。',
    
    '象辞的意思是：上卦为云，下卦为天，天上有云便是需卦的卦象。君子从中受到启示，要饮食宴乐。',
    '象辞的意思是：在郊外等待，是不冒险前进。利于持恒，没有灾难，是没有失去正常的理智。',
    '象辞的意思是：在沙滩上等待，是因九二爻宽厚而居中，虽然有些口舌，但最终是吉祥的。',
    '象辞的意思是：在淤泥中等待，是因为外边有灾难。自己招来贼寇，是告诫人们要恭敬谨慎才不会陷于失败。',
    '象辞的意思是：在忧患中等待，是柔顺而听从才导致大难不死。',
    '象辞的意思是：酒食宴乐中的吉祥，是因为九五爻阳爻居中的缘故。',
    '象辞的意思是：不速之客的来到，尊敬他们而最终吉祥；上六虽然位置不当，但没有大的损失。']
len(xiangci), len(xiangci_fanyi)

(30, 35)

In [5]:
tuanci_fanyi = [
    "多么伟大啊，阳气的始生！万物因它而生，乃至天体也被它控制着（即天气因阳生而转暖）。它使天空布云施雨，（在雨水的滋润下）万物开始了生长，在大地上显露出了本来的形象；太阳反复运行，使六个时辰处于美好的白天；乾卦的六爻，控制着天体的寒来暑往。天体的运行规律，赋予万物不同的性质和寿命，经常保持阴阳的均衡和合，才是利于坚守的正道。阳气不但使万物拥有了生命，而且使万国都因此安宁。",
    "至大无际啊，坤元的始生！万物都借助你得以生成，你顺应着秉承着天道。大地深厚负载万物，品德博大没有边际。包容无限而广大，各类事物都因你而亨通。牝马属于地上的动物，驰骋四野没有疆界。阴柔温顺利于正道。君子行动的开始会迷失方向，随后便会掌握行动的常规法则。西南得朋，是能够见到自己的同类；东北丧朋，却会得到结婚的喜庆。安于正道的吉祥，是与无边无际的地道相应的。",
    '彖辞的意思是：屯卦，刚柔开始交流而出生艰难。上卦坎为险，下卦震为动，在险难中运动求得生存，会得到大的亨通并且是正道。震又为雷，坎又为雨，所以打雷下雨使天地满盈。天地草创，适于建立侯王功业，因为天下还处于不安宁的状态中。',
    '彖辞的意思是：蒙卦，上卦为艮，艮为山为止；下卦为坎，坎为水为险。所以说山下有险，遇险而止，这就是蒙卦的意思。蒙卦亨通，是由于顺应时序和中庸的原则。不是我去求蒙童，而是蒙童求我，这样蒙童的心志才能与我的心志相应。初次占筮可以告诉他，是因为九二阳爻处于下卦之中；两次三次地占筮是对神的亵渎，则不能把结果告诉他，因为他因蒙昧而亵渎神灵。要用启蒙的方法培养其纯正无邪的品质，这是圣人的功业。',
    '彖辞的意思是：需就是等待的意思。为什么要等待？因为有险难在前面。性格刚健就不会陷于险难中，其象征的意义是不会困于穷途末路。需卦有诚信，光明而亨通，守正则吉祥。这是由于九五爻居于天子之位，既居中又居正位的缘故。所以利于涉过大川险阻，前进会建立功业。'
]
len(tuanci_fanyi)

5

In [6]:
guaci_fanyi = [ctt for idx, ctt in enumerate(jingwen_fanyi) if not(idx % 7)]    
gua_xiangci_fanyi = [ctt for idx, ctt in enumerate(xiangci_fanyi) if not(idx % 7)]

tbls = [
    pd.DataFrame(
        {
            # "卦名": multiply(gua),
            "卦辞翻译": multiply(guaci_fanyi),
            "彖辞翻译": multiply(tuanci_fanyi),
            "象辞翻译": multiply(gua_xiangci_fanyi),
            "爻辞": yaoci,
            "爻辞翻译": [ctt for idx, ctt in enumerate(jingwen_fanyi) if idx % 7],
            "小象辞": xiangci,
            "小象辞翻译": [ctt for idx, ctt in enumerate(xiangci_fanyi) if idx % 7],
        }
    )
]

In [7]:
soups = []
for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        soups.append(
            BeautifulSoup(item.get_content(), 'html')
        )

/var/folders/s1/1jpfx0m52rj4k7cgqkh7g3q40000gn/T/ipykernel_55563/2686135823.py:5: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  BeautifulSoup(item.get_content(), 'html')


In [8]:
for x in [8,9, 10,11,12, 13, 14,]:
    tbls.append(
        parse_soup(soups[x])
    ) 
df_new = pd.concat(tbls).reset_index(drop=True)
# df_new.to_csv("易经.csv", index=False)

36 36 42 42 6
36
54 54 63 63 9
54
60 60 70 70 10
60
60 60 70 70 10
60
60 60 70 70 10
60
42 42 49 49 7
42
42 42 49 49 7
42


In [9]:
## 最后的文档应该是6*64=384行。

In [10]:
df_ori = pd.concat(
    [
        pd.read_csv("/Users/minkexiu/Desktop/工作簿1.csv").dropna().reset_index(drop=True),
        pd.read_csv("/Users/minkexiu/Desktop/工作簿2.csv").dropna().reset_index(drop=True)
    ]
).reset_index(drop=True)

In [11]:
df_ori

,卦序,卦名,象辞,卦辞,彖辞
0,1.0,乾卦 （下乾上乾，乾为天）,天行健，君子以自强不息,乾。元，亨，利，贞。 （用九：见群龙无首，吉。）,大哉乾元，万物资始，乃统天。云行雨施，品物流形。大明终始，六位时成。时乘六龙以御天。乾道变化...
1,2.0,坤卦 （下坤上坤，坤为地）,地势坤，君子以厚德载物。,坤。元亨，利牝马之贞。君子有攸往，先迷后得主，利；西南得朋，东北丧朋，安贞吉。 （用六： 利...,至哉坤元，万物资生，乃顺承天。坤厚载物，德合无疆。含弘光大，品物咸亨。\n牝马地类，行地无疆...
2,3.0,屯卦 （下震上坎，水雷屯）,云雷屯，君子以经纶。,屯。元亨，利贞，勿用有攸往。利建侯。,屯，刚柔始交而难生。动乎险中，大、亨、贞。雷雨之动满盈，天造草昧，宜建侯而不宁。
3,4.0,蒙卦 （下坎上艮，山水蒙）,山下出泉，蒙；君子以果行育德。,蒙。亨。匪我求童蒙，童蒙求我。初筮告，再三渎，渎则不告。利贞。,蒙，山下有险，险而止，蒙。蒙亨，以亨行时中也。\n匪我求童蒙，童蒙求我，志应也。初筮告，以刚...
4,5.0,需卦 （下乾上坎，水天需）,云上于天，需；君子以饮食宴乐。,需。有孚，光亨，贞吉；利涉大川。,需，须也，险在前也。刚健而不陷，其义不困穷矣。\n需，有孚，光亨，贞吉，位乎天位，以正中也。...
...,...,...,...,...,...
59,60.0,节卦（兑下坎上，水泽节）,泽上有水，节；君子以制数度，议德行。,节。亨；苦节，不可贞。,节亨，刚柔他，而刚得中；苦节不可贞，其道穷也。说以得险，当位以节，中正以通，天地节而四时成。...
60,61.0,中孚卦（兑下巽上，风泽中孚）,泽上有风，中孚；君子以议狱缓死。,中孚。豚鱼吉，利涉大川，利贞。,中孚，柔在内而刚得中；说而巽，孚乃化邦也。豚鱼吉，信及豚鱼也；利涉大川，乘木舟虚也；中孚以利...
61,62.0,小过卦（艮下震上，雷山小过）,山上有雷，小过；君子以行过乎恭，丧过乎哀，用过乎俭。,小过。亨，利贞，可小事，不可大事。飞鸟遗之音，不宜上，宜下，大吉。,小过，小者过而亨也；过以利贞，与时行也；柔得中，是以小事吉也；刚失位而不中，是以不可大事也；...
62,63.0,既济卦（离下坎上，水火既济）,水在火上，既济；君子以思患而预防之。,既济。亨小，利贞；初吉，终乱。,既济亨，小者亨也；利贞，刚柔正而位当也；初吉，柔得中也；终止则乱，其道穷也。


In [12]:
df_new["卦序"] = multiply(list(range(1, 64+1)))

In [13]:
df_new

,卦辞翻译,彖辞翻译,象辞翻译,爻辞,爻辞翻译,小象辞,小象辞翻译,卦序
0,【古解作】元，始也；亨，通也；利，和也；贞，正也。【今解作】当你满怀信心开始做一件事情时，谁...,多么伟大啊，阳气的始生！万物因它而生，乃至天体也被它控制着（即天气因阳生而转暖）。它使天空布...,（大）象译文：天体运行刚健而永不停息，君子应当效法天道，自强不息。,初九曰 ：“潜龙勿用。”,这句经文的意思是：“身居下位，时机还没有成熟，所以应当像潜藏的龙一样不要施展你的才干。”,（小）象曰：潜龙勿用，阳在下也。,初九之所以“潜龙勿用”，是因为此阳爻的位置处于最下面的位置。,1
1,【古解作】元，始也；亨，通也；利，和也；贞，正也。【今解作】当你满怀信心开始做一件事情时，谁...,多么伟大啊，阳气的始生！万物因它而生，乃至天体也被它控制着（即天气因阳生而转暖）。它使天空布...,（大）象译文：天体运行刚健而永不停息，君子应当效法天道，自强不息。,九二曰 ：“见龙在田，利见大人。”,此句经文的意思是：“龙出现在田野之上，有利于见到大人物来发展自己。”,见龙在田，德施普也。,九二的“见龙在田”，指的是九二阳爻已来到地上，可以发挥它的养生万物之德了。,1
2,【古解作】元，始也；亨，通也；利，和也；贞，正也。【今解作】当你满怀信心开始做一件事情时，谁...,多么伟大啊，阳气的始生！万物因它而生，乃至天体也被它控制着（即天气因阳生而转暖）。它使天空布...,（大）象译文：天体运行刚健而永不停息，君子应当效法天道，自强不息。,九三曰 ：“君子终日乾乾，夕惕若厉，无咎。”,这句经文的意思是：“君子终日自强不息，每天晚上对自己进行深刻反省，保持警惕的心，检查自己的失...,终日乾乾，反复道也。,九三的“终日乾乾”，即反复运行，永不停息的意思。,1
3,【古解作】元，始也；亨，通也；利，和也；贞，正也。【今解作】当你满怀信心开始做一件事情时，谁...,多么伟大啊，阳气的始生！万物因它而生，乃至天体也被它控制着（即天气因阳生而转暖）。它使天空布...,（大）象译文：天体运行刚健而永不停息，君子应当效法天道，自强不息。,九四： “或跃在渊，无咎。”,这句经文的意思是：“根据不同时机，可以一跃九天，也可以复沉于渊，都不会有什么灾难。”,或跃在渊，进无咎也。,九四的“或跃在渊”，是说前进（或归隐）不会有灾难的意思。,1
4,【古解作】元，始也；亨，通也；利，和也；贞，正也。【今解作】当你满怀信心开始做一件事情时，谁...,多么伟大啊，阳气的始生！万物因它而生，乃至天体也被它控制着（即天气因阳生而转暖）。它使天空布...,（大）象译文：天体运行刚健而永不停息，君子应当效法天道，自强不息。,九五曰 ：“飞龙在天，利见大人。”,这一句经文的意思是：“飞龙遨游于天际，有利于见到大人物。”,飞龙在天，大人造也。,九五的“飞龙在天”，是指大人物将会出现而大有作为的意思。,1
...,...,...,...,...,...,...,...,...
379,经文意思是：未济卦，亨通，小狐狸快要能够走过小河，沾湿了它的尾巴，无所利。,彖辞的意思是：未济卦的亨通，是由于六五爻以柔顺居中。小狐狸过河，是还没有走出河中。沾湿了尾巴...,象辞的意思是：火在水上便是未济卦的卦象。君子从卦象中受到启发，谨慎分辨各种事物，选择好自己居...,九二： 曳其轮，贞吉。,经文意思是：拉着车轮，守正道吉祥。,象曰：九二贞吉，中以行正也。,象辞的意思是：九二爻守正道吉祥，是居中而行正道。,64
380,经文意思是：未济卦，亨通，小狐狸快要能够走过小河，沾湿了它的尾巴，无所利。,彖辞的意思是：未济卦的亨通，是由于六五爻以柔顺居中。小狐狸过河，是还没有走出河中。沾湿了尾巴...,象辞的意思是：火在水上便是未济卦的卦象。君子从卦象中受到启发，谨慎分辨各种事物，选择好自己居...,六三： 未济，征凶，利涉大川。,经文意思是：没有渡过河，征伐有凶险，有利于跋涉大川。,象曰：未济征凶，位不当也。,象辞的意思是：没有渡过河，征伐有凶险，是由于六三的位置不当。,64
381,经文意思是：未济卦，亨通，小狐狸快要能够走过小河，沾湿了它的尾巴，无所利。,彖辞的意思是：未济卦的亨通，是由于六五爻以柔顺居中。小狐狸过河，是还没有走出河中。沾湿了尾巴...,象辞的意思是：火在水上便是未济卦的卦象。君子从卦象中受到启发，谨慎分辨各种事物，选择好自己居...,九四： 贞吉，悔亡，震用伐鬼方，三年有赏于大国。,经文意思是：守正道吉祥，没有悔恨，振奋起来征伐鬼方国，三年获得成功，得到大国的奖赏。,象曰：贞吉悔亡，志行也。,象辞的意思是：守正道吉祥，没有悔恨，是由于九四爻志在必行。,64
382,经文意思是：未济卦，亨通，小狐狸快要能够走过小河，沾湿了它的尾巴，无所利。,彖辞的意思是：未济卦的亨通，是由于六五爻以柔顺居中。小狐狸过河，是还没有走出河中。沾湿了尾巴...,象辞的意思是：火在水上便是未济卦的卦象。君子从卦象中受到启发，谨慎分辨各种事物，选择好自己居...,六五：贞吉，无悔，君子之光，有孚，吉。,经文意思是：守正道吉祥，没有悔恨，君子的光耀，有诚信，吉祥。,象曰：君子之光，其晖吉也。,象辞的意思是：君子的光耀，是说在他的光彩照耀下带来了吉祥。,64


In [14]:
df = df_new.merge(
    df_ori, how="inner", on="卦序"
).rename(columns={"象辞翻译": "大象辞翻译", "象辞": "大象辞"})[["卦序", "卦名", "卦辞", "卦辞翻译", "彖辞", "彖辞翻译", "大象辞", "大象辞翻译", "爻辞", "爻辞翻译", "小象辞", "小象辞翻译"]]

In [15]:
# 初始化一个新的DataFrame来存储结果  
reversed_df = pd.DataFrame(columns=df.columns)  
  
# 遍历DataFrame，每次处理6行  
for i in range(0, len(df), 6):  
    # 检查我们是否还有足够的行来进行倒序（如果DataFrame的行数不是6的倍数）  
    end_index = min(i + 6, len(df))  
    subset = df[i:end_index]  
    # 将子集倒序  
    reversed_subset = subset[::-1]  
    # 将倒序的子集追加到新的DataFrame中  
    reversed_df = pd.concat([reversed_df, reversed_subset], ignore_index=True)  
  
# 现在reversed_df包含了原始DataFrame的每6行倒序排列的结果  
reversed_df

,卦序,卦名,卦辞,卦辞翻译,彖辞,彖辞翻译,大象辞,大象辞翻译,爻辞,爻辞翻译,小象辞,小象辞翻译
0,1,乾卦 （下乾上乾，乾为天）,乾。元，亨，利，贞。 （用九：见群龙无首，吉。）,【古解作】元，始也；亨，通也；利，和也；贞，正也。【今解作】当你满怀信心开始做一件事情时，谁...,大哉乾元，万物资始，乃统天。云行雨施，品物流形。大明终始，六位时成。时乘六龙以御天。乾道变化...,多么伟大啊，阳气的始生！万物因它而生，乃至天体也被它控制着（即天气因阳生而转暖）。它使天空布...,天行健，君子以自强不息,（大）象译文：天体运行刚健而永不停息，君子应当效法天道，自强不息。,上九曰 ：“亢龙有悔。”【用九】见群龙无首，吉。,这一句经文的意思是：“龙向上飞得太高了，便会有后悔的事情发生。”【用九】出现一群龙而没有首领...,亢龙有悔，盈不可久也。【用九】用九，天德不可为首也。,上九的“亢龙有悔”，是说物不可以终盛，会盛极而衰，阳极而阴生之意。【用九】阳刚之德不会自以为...
1,1,乾卦 （下乾上乾，乾为天）,乾。元，亨，利，贞。 （用九：见群龙无首，吉。）,【古解作】元，始也；亨，通也；利，和也；贞，正也。【今解作】当你满怀信心开始做一件事情时，谁...,大哉乾元，万物资始，乃统天。云行雨施，品物流形。大明终始，六位时成。时乘六龙以御天。乾道变化...,多么伟大啊，阳气的始生！万物因它而生，乃至天体也被它控制着（即天气因阳生而转暖）。它使天空布...,天行健，君子以自强不息,（大）象译文：天体运行刚健而永不停息，君子应当效法天道，自强不息。,九五曰 ：“飞龙在天，利见大人。”,这一句经文的意思是：“飞龙遨游于天际，有利于见到大人物。”,飞龙在天，大人造也。,九五的“飞龙在天”，是指大人物将会出现而大有作为的意思。
2,1,乾卦 （下乾上乾，乾为天）,乾。元，亨，利，贞。 （用九：见群龙无首，吉。）,【古解作】元，始也；亨，通也；利，和也；贞，正也。【今解作】当你满怀信心开始做一件事情时，谁...,大哉乾元，万物资始，乃统天。云行雨施，品物流形。大明终始，六位时成。时乘六龙以御天。乾道变化...,多么伟大啊，阳气的始生！万物因它而生，乃至天体也被它控制着（即天气因阳生而转暖）。它使天空布...,天行健，君子以自强不息,（大）象译文：天体运行刚健而永不停息，君子应当效法天道，自强不息。,九四： “或跃在渊，无咎。”,这句经文的意思是：“根据不同时机，可以一跃九天，也可以复沉于渊，都不会有什么灾难。”,或跃在渊，进无咎也。,九四的“或跃在渊”，是说前进（或归隐）不会有灾难的意思。
3,1,乾卦 （下乾上乾，乾为天）,乾。元，亨，利，贞。 （用九：见群龙无首，吉。）,【古解作】元，始也；亨，通也；利，和也；贞，正也。【今解作】当你满怀信心开始做一件事情时，谁...,大哉乾元，万物资始，乃统天。云行雨施，品物流形。大明终始，六位时成。时乘六龙以御天。乾道变化...,多么伟大啊，阳气的始生！万物因它而生，乃至天体也被它控制着（即天气因阳生而转暖）。它使天空布...,天行健，君子以自强不息,（大）象译文：天体运行刚健而永不停息，君子应当效法天道，自强不息。,九三曰 ：“君子终日乾乾，夕惕若厉，无咎。”,这句经文的意思是：“君子终日自强不息，每天晚上对自己进行深刻反省，保持警惕的心，检查自己的失...,终日乾乾，反复道也。,九三的“终日乾乾”，即反复运行，永不停息的意思。
4,1,乾卦 （下乾上乾，乾为天）,乾。元，亨，利，贞。 （用九：见群龙无首，吉。）,【古解作】元，始也；亨，通也；利，和也；贞，正也。【今解作】当你满怀信心开始做一件事情时，谁...,大哉乾元，万物资始，乃统天。云行雨施，品物流形。大明终始，六位时成。时乘六龙以御天。乾道变化...,多么伟大啊，阳气的始生！万物因它而生，乃至天体也被它控制着（即天气因阳生而转暖）。它使天空布...,天行健，君子以自强不息,（大）象译文：天体运行刚健而永不停息，君子应当效法天道，自强不息。,九二曰 ：“见龙在田，利见大人。”,此句经文的意思是：“龙出现在田野之上，有利于见到大人物来发展自己。”,见龙在田，德施普也。,九二的“见龙在田”，指的是九二阳爻已来到地上，可以发挥它的养生万物之德了。
...,...,...,...,...,...,...,...,...,...,...,...,...
379,64,未济卦（坎下离上，火水未济）,未济。亨，小狐汔济，濡其尾，无攸利。,经文意思是：未济卦，亨通，小狐狸快要能够走过小河，沾湿了它的尾巴，无所利。,未济亨，柔得中也；小狐汔济，未出中也；濡其尾，无攸利，不续终也；虽不当位，刚柔应也。,彖辞的意思是：未济卦的亨通，是由于六五爻以柔顺居中。小狐狸过河，是还没有走出河中。沾湿了尾巴...,火在水上，未济；君子以慎辨物居方。,象辞的意思是：火在水上便是未济卦的卦象。君子从卦象中受到启发，谨慎分辨各种事物，选择好自己居...,六五：贞吉，无悔，君子之光，有孚，吉。,经文意思是：守正道吉祥，没有悔恨，君子的光耀，有诚信，吉祥。,象曰：君子之光，其晖吉也。,象辞的意思是：君子的光耀，是说在他的光彩照耀下带来了吉祥。
380,64,未济卦（坎下离上，火水未济）,未济。亨，小狐汔济，濡其尾，无攸利。,经文意思是：未济卦，亨通，小狐狸快要能够走过小河，沾湿了它的尾巴，无所利。,未济亨，柔得中也；小狐汔济，未出中也；濡其尾，无攸利，不续终也；虽不当位，刚柔应也。,彖辞的意思是：未济卦的亨通，是由于六五爻以柔顺居中。小狐狸过河，是还没有走出河中。沾湿了尾巴...,火在水上，未济；君子以慎辨物居方。,象辞的意思是：火在水上便是未济卦的卦象。君子从卦象中受到启发，谨慎分辨各种事物，选择好自己居...,九四： 贞吉，悔亡，震用伐鬼方，三年有赏于大国。,经文意思是：守正道吉祥，没有悔恨，振奋起来征伐鬼方国，三年获得成功，得到大国的奖赏。,象曰：贞吉悔亡，志行也。,象辞的意思是：守正道吉祥，没有悔恨，是由于九四爻志在必行。
381,64,未济卦（坎下离上，火水未济）,未济。亨，小狐汔济，濡其尾，无攸利。,经文意思是：未济卦，亨通，小狐狸快要能够走过小河，沾湿了它的尾巴，无所利。,未济亨，柔得中也；小狐汔济，未出中也；濡其尾，无攸利，不续终也；虽不当位，刚柔应也。,彖辞的意思是：未济卦的亨通，是由于六五爻以柔顺居中。小狐狸过河，是还没有走出河中。沾湿了尾巴...,火在水上，未济；君子以慎辨物居方。,象辞的意思是：火在水上便是未济卦的卦象。君子从卦象中受到启发，谨慎分辨各种事物，选择好自己居...,六三： 未济，征凶，利涉大川。,经文意思是：没有渡过河，征伐有凶险，有利于跋涉大川。,象曰：未济征凶，位不当也。,象辞的意思是：没有渡过河，征伐有凶险，是由于六三的位置不当。
382,64,未济卦（坎下离上，火水未济）,未济。亨，小狐汔济，濡其尾，无攸利。,经文意思是：未济卦，亨通，小狐狸快要能够走过小河，沾湿了它的尾巴，无所利。,未济亨，柔得中也；小狐汔济，未出中也；濡其尾，无攸利，不续终也；虽不当位，刚柔应也。,彖辞的意思是：未济卦的亨通，是由于六五爻以柔顺居中。小狐狸过河，是还没有走出河中。沾湿了尾巴...,火在水上，未济；君子以慎辨物居方。,象辞的意思是：火在水上便是未济卦的卦象。君子从卦象中受到启发，谨慎分辨各种事物，选择好自己居...,九二： 曳其轮，贞吉。,经文意思是：拉着车轮，守正道吉祥。,象曰：九二贞吉，中以行正也。,象辞的意思是：九二爻守正道吉祥，是居中而行正道。


In [16]:
reversed_df["真卦名"] = reversed_df.卦名.apply(lambda x: x.split("（")[0])
reversed_df["卦型"] = reversed_df.卦名.apply(lambda x: x.split("（")[1].replace("）", "").split("，")[0])
reversed_df["称谓"] = reversed_df.卦名.apply(lambda x: x.split("（")[1].replace("）", "").split("，")[1])

In [17]:
gxfx = pd.DataFrame(
    {
        "type":reversed_df.卦型.value_counts().index
    }
)

In [18]:
gxfx["left"] = gxfx.type.str[:2]
gxfx["right"] = gxfx.type.str[2:]

In [19]:
def process_up(x):
    left = x["left"]
    right = x["right"]
    if "上" in left:
        return left.replace("上", "")
    else: 
        return right.replace("上", "")

def process_down(x):
    left = x["left"]
    right = x["right"]
    if "上" in left:
        return right.replace("下", "") 
    else: 
        return left.replace("下", "")

    
gxfx["上卦"] = gxfx.apply(process_up, axis=1)
gxfx["下卦"] = gxfx.apply(process_down, axis=1)

In [20]:
# gxfx.to_csv("gxfx.csv", index=False)

In [21]:
graph = pd.DataFrame(
    {
        "卦": ["乾", "兑","离","震","巽","坎","艮","坤"],
        "图": ["☰","☱","☲","☳","☴","☵","☶","☷"]


    }
)

In [22]:
gxfx = gxfx.merge(
    graph.rename(columns={
        "卦": "上卦",
        "图": "上图"
    })
)

In [23]:
gxfx = gxfx.merge(
    graph.rename(columns={
        "卦": "下卦",
        "图": "下图"
    })
)

In [24]:
gxfx["尝试"] = gxfx.apply(lambda x: x["上图"] + "<br>" + x["下图"], axis=1)

In [25]:
reversed_df = reversed_df.drop(columns=["卦名"]).rename(columns={"真卦名": "卦名"})

In [26]:
reversed_df = reversed_df.merge(
    gxfx[["type", "尝试"]].rename(columns={"type": "卦型", "尝试": "卦图"})
)

In [27]:
with open("简易易经.html", "w") as f:
    f.write(reversed_df[
    ['卦序', '卦名', '卦型', '称谓', '卦图', '卦辞', '卦辞翻译', '彖辞', '彖辞翻译', '大象辞', '大象辞翻译', '爻辞', '爻辞翻译', '小象辞','小象辞翻译', ]
].to_html(index=False).replace("&lt;br&gt;", "<br>"))